# Create Final AAL Results File
This notebook reads the results of the AAL Calculator from s3 and joins them together to create a final AAL results csv file for an entire project area.

In order for this notebook to be run successfully, all of the results of the AAL Calculator must be in the right location on s3.

User inputs:
 - project (e.g. `'DC'`)
 - book (e.g. `'Uniform'`)
 
Author: Alec Brazeau - abrazeau@dewberry.com

In [ ]:
import pandas as pd
import sys; sys.path.append('../core')
from s3utils import *
%matplotlib inline

## User specified parameters

In [ ]:
if TRI:
    aal_files = s3List('pfra', 'RiskAssessment/{}/Results/TRI'.format(project), 'AAL', '.csv')
else:
    aal_files = [x for x in s3List('pfra', 'RiskAssessment/{}/Results'.format(project), 'AAL', '.csv') if not 'TRI' in x]
s3files = [x for x in aal_files if book in x and 'Final' not in x and 'Losses' not in x and 'anoms' not in x]
s3files = [f for f in s3files if not '/A/' in f and not '/B/' in f]
print('Files to be joined:')
for x in s3files: print(x)

In [ ]:
plus_codes = []
# read all points, make a set of unique plus codes from all
# Make attribute tables from current csvs
for f in s3files:
    temp = pd.read_csv(f)
    for idx in temp['plus_code']:
        plus_codes.append(idx)
        
idx = list(set(plus_codes))
print('Total points in {} {}: {}'.format(project, book, len(idx)))
fluvial_joined = pd.DataFrame(index=idx)
pluvial_joined = pd.DataFrame(index=idx)

## Join Fluvial AAL Results First

In [ ]:
# Cycle through fluvials
fluv_s3files = [x for x in s3files if 'F0' in x]
for f in fluv_s3files:
    if TRI:
        model = os.path.basename(f).split('_')[-3]
    else:
        model = os.path.basename(f).split('_')[-2]
    df = pd.read_csv(f)
    df.rename(columns={'AAL':'{}_AAL'.format(model)}, inplace=True)
    df.set_index('plus_code', inplace=True)
    fluvial_joined = fluvial_joined.join(df, how='left')

In [ ]:
fluvial_joined['FLUV_AAL'] = fluvial_joined.sum(axis=1)
ax = fluvial_joined['FLUV_AAL'].plot(figsize=(32,4))
ax.set_ylim(0,1000)

## Join Pluvial AAL Results

In [ ]:
pluv_s3files = [x for x in s3files if 'P0' in x]
for p in pluv_s3files:
    if TRI:
        model = os.path.basename(p).split('_')[-3]
    else:
        model = os.path.basename(p).split('_')[-2]
    dp = pd.read_csv(p)
    dp.rename(columns={'AAL':'{}_AAL'.format(model)}, inplace=True)
    dp.set_index('plus_code', inplace=True)
    pluvial_joined = pluvial_joined.join(dp, how='left')

### Test to see if any row has more than one positive value... if so, defer to the last column that is not zero

In [ ]:
pluvial_joined = pluvial_joined.fillna(0)
## This will set all pluvial AAL columns to 0 except for the last one.
pluv_dups = [1]
while len(pluv_dups) > 0:
    # check for dups, append the plus_code to a list
    pluv_dups = []
    for row in list(zip(pluvial_joined.index, map(set, pluvial_joined.values))):
        if len(row[1]) > 2:
            pluv_dups.append(row[0])
    print('plus_codes with more than one pluvial AAL:', pluv_dups)
    # set the aal = to zero except for the last one
    for pcode in pluv_dups:
        for i, col in enumerate(pluvial_joined.columns.tolist()):
            if pluvial_joined.loc[pcode, col] != 0:  # we are deffering to the last model with a value
                pluvial_joined.loc[pcode, col] = 0
                break

In [ ]:
# Get one pluvial aal for each point
pluvial_joined['PLUV_AAL'] = pluvial_joined.sum(axis=1)
ax = pluvial_joined['PLUV_AAL'].plot(figsize=(32,4))
ax.set_ylim(0,1000)

## Join the fluvial and pluvial results to get a final AAL dataframe

In [ ]:
final_product = pd.DataFrame(pluvial_joined['PLUV_AAL']).join(pd.DataFrame(fluvial_joined['FLUV_AAL']), how='outer')
final_product['TOT_AAL'] = final_product.sum(axis=1)

In [ ]:
ax = final_product['TOT_AAL'].plot(figsize=(32,4))
ax.set_ylim(0,3000)

In [ ]:
final_product.max()

In [ ]:
final_product.shape

## Join the building attributes and ground elevation to the final AAL table

In [ ]:
# start with a dataframe of only the idxs
idx_only = final_product.copy().drop(columns=final_product.columns.tolist())

temp_dfs = []
for f in s3files:
    # read the attribute files
    if TRI:
        attrs_f = f.replace('Results/TRI', 'Attributes').replace('AAL_', '').replace('_TRI', '')
    else:
        attrs_f = f.replace('Results', 'Attributes').replace('AAL_', '')
    attr_df = pd.read_csv(attrs_f).set_index('plus_code').drop(columns=['geom', 'damage_code'])
    
    # drop the shp_nam column that is only in some csvs
    try:
        attr_df = attr_df.drop(columns=['shp_nam'])
    except KeyError as e:
        print(e, 'for {}'.format(attrs_f))
    
    # get the ground elevations. These are only available in the WSE files
    wse_f = f.replace('AAL', 'WSE')
    groundelev = pd.read_csv(wse_f).set_index('plus_code')[['GroundElev']]
    
    # join the dataframes together and append to a list for concatenation
    attr_temp = idx_only.join(attr_df, how='inner')
    ge_temp = idx_only.join(groundelev, how='inner')
    temp = attr_temp.join(ge_temp)
    temp_dfs.append(temp)

# concatenate the data into one dataframe, dropping duplicates
attr_data = pd.concat(temp_dfs).drop_duplicates()
# drop rows that have duplicate indices, keeping the first row
# these dups happen when ground elevations are different between models
attr_data = attr_data.loc[~attr_data.index.duplicated(keep='first')]

## Join the attribute data to the final AAL dataframe

In [ ]:
final_data_product = final_product.join(attr_data, how='inner')
final_data_product.index.name = 'plus_code'
final_data_product['Excluded'] = None # leave this as null for now...
if 'pfra_cat' in list(final_data_product.columns):
    final_data_product = final_data_product.drop(columns=['pfra_cat'])
print('Final shape: {}'.format(final_data_product.shape))
final_data_product.head(3)

## Write the final table to a file

In [ ]:
# write the file
final_fn = 'Final_AAL_{0}_{1}{2}.csv'.format(project, book, TRI)
final_path = os.path.join(output_dir, final_fn)
final_data_product.to_csv(final_path)
print('Result written: {}'.format(final_path))

# END